In [11]:
import menpo.io as mio
import os
from sklearn import svm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import Models
from Models import ChangeVector
path_to_svm_training_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'


def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

#process changingVector, reduce dimension from 2x68 to 1x68, by process PCA
def pca(changeVector):
    X = np.array(changeVector)
    pca_model = PCA(n_components=1)
    return pca_model.fit_transform(X)


training_images = mio.import_images(path_to_svm_training_database, verbose=True)
training_images = training_images.map(process)

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'


    


Found 1146 assets, index the returned LazyList to import.


In [42]:
def process_center_point_landmark(landmark):
    x_array = []
    y_array = []
    xmean = 0
    ymeam = 0
    for p in landmark:
        x_array.append(p[0])
        y_array.append(p[1])
#     return x_array, y_array
    xmean = np.mean(x_array)
    ymean = np.mean(y_array)
    center_point_landmark = []
    for i in range(0,68):
        center_point_landmark.append([x_array[i] - float(xmean), y_array[i] - float(ymean)])
    return center_point_landmark
#     return x_array, y_array

In [45]:
center_vector = process_center_point_landmark(training_images[0].landmarks['PTS'].lms.points)
len(center_vector)

68

In [ ]:
#create training data
count = 0;
svm_training_data = []
while(count < len(training_images)):
    file_path = str(training_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            if(int(float(x)) not in data_facs and int(float(x))!= 0):
                data_facs.append(int(float(x)))
    data_facs.sort()
    fi.close()
    
    centerPointVector = []
    landmark_neutral = training_images[count].landmarks['PTS'].lms.points
    landmark_perk = training_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])

    svm_training_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   
    count = count + 2
#     centerPointVector = []

In [7]:
svm_training_data[1].landmarkChange

[[5.1327602383554165, 1.2203631977513041],
 [5.2732395835818693, 0.5976734021072474],
 [5.7836381695654495, -1.493530994023466],
 [5.9913809122068358, -3.4622476395723254],
 [5.6928822032760991, -1.8257387066762547],
 [6.0037766257092642, -1.3563880735408702],
 [5.8456501617148717, -1.6024899613446735],
 [3.0157845290365231, 0.30828467057152409],
 [1.037023502789765, 0.7524142580808757],
 [3.4537833081116389, 0.84602716007799472],
 [6.5144219058381339, 2.0723680636354516],
 [6.2323295041791482, 1.3195099608449254],
 [6.7862623277351872, 2.5956931379639627],
 [8.1269712907822225, 5.194668618523707],
 [8.2304237750317668, 2.0759240669850101],
 [8.0950095482428281, -1.2172992008063943],
 [8.4028370784210153, -1.8715780464548573],
 [5.3084011112767513, 6.9893969497290129],
 [2.9938607929542087, 4.553139573807627],
 [2.6021957262700539, 3.8852858060456299],
 [1.9444045852477885, 3.6295562676563122],
 [4.2112589979640447, 1.8092797500339657],
 [5.433265146658151, -0.56067563723053127],
 [2.0

In [ ]:

facs = []
#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if(int(facs_code) not in facs and int(facs_code)!= 0):
            facs.append(facs_code)
facs.sort()

#create model for each action unit in facs[]
models = []
au_models_score = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_training_data:
        if(au in data.facs):
            y_label.append(1)
        else:
            y_label.append(0)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmarkChange:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    clf = svm.LinearSVC()
    clf.fit(x_training, y_label)
    au_models_score.append(clf.score(x_training, y_label))
    models.append(clf)
    
#create testing data
svm_testing_data = []
path_to_svm_testing_database = "/Programing/GR/Code/CK+/test-aam-images/**/**/**/*"
testing_images = mio.import_images(path_to_svm_testing_database, verbose=True)
testing_images = testing_images.map(process)

count = 0;
while(count < len(testing_images)):
    file_path = str(testing_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            if(int(float(x)) not in data_facs and int(float(x)) != 0):
                data_facs.append(int(float(x)))
    #print(array)
    fi.close()
    
    landmarkChange = []
    landmark_neutral = testing_images[count].landmarks['PTS'].lms.points
    landmark_perk = testing_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
    svm_testing_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   
    count = count + 2

    
#evaluate trained model with test data and get score
print('#######')
print('Sccore: ')
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_testing_data:
        if(au in data.facs):
            y_label.append(1)
        else:
            y_label.append(0)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmarkChange:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    print(models[facs.index(au)].score(x_training, y_label))